In [1]:
import sys

import gymnasium as gym

sys.path.append("..")

In [2]:
from training.dqnetwork import DQNetwork

In [3]:
env = gym.make("CartPole-v1", render_mode="rgb_array")

In [4]:
from utils.reproducibility import set_global_seed

set_global_seed(seed=42, env=env, tensorflow_deterministic=True, verbose=True)

2025-10-20 16:32:22,907 — INFO — [set_global_seed] TensorFlow seed set.
2025-10-20 16:32:22,909 — INFO — [set_global_seed] Gymnasium environment seed set.
2025-10-20 16:32:22,909 — INFO — [set_global_seed] All available seeds set to 42.


In [5]:
neuron_count_per_hidden_layer = [64, 32]

action_net = DQNetwork(
    env.observation_space.shape,
    neuron_count_per_hidden_layer,
    env.action_space.n,
    summary=True,
)
target_net = DQNetwork(
    env.observation_space.shape,
    neuron_count_per_hidden_layer,
    env.action_space.n,
    summary=True,
)

target_net.model.set_weights(action_net.model.get_weights())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output (Dense)                  │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,466 (9.63 KB)

 Trainable params: 2,466 (9.63 KB)

 Non-trainable params: 0 (0.00 B)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 64)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output (Dense)                  │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,466 (9.63 KB)

 Trainable params: 2,466 (9.63 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
episode_count = 500  # Total number of training episodes
episode_max_steps = 400  # Maximum number of steps per episode

replay_memory_max_size = (
    100000  # Maximum number of transitions stored into the replay memory
)
replay_memory_init_size = (
    1000  # Maximum number of transitions stored into the replay memory
)
batch_size = 64  # Mini-batch size

step_per_update = 4  # Number of total steps executed between successive updates of the action model weights
step_per_update_target_model = 8  # Number of total steps executed between successive replaces of the target model weights

max_epsilon = 1.0  # Exploration probability at start
min_epsilon = 0.01  # Minimum exploration probability
epsilon_decay = 0.0002  # Decay for exploration probability

gamma = 0.99  # Discount factor

moving_avg_window_size = 20  # Number of consecutive episodes to be considered in the calculation of the total reward moving average
moving_avg_stop_thr = 100

In [7]:
from agent.dqagent import DQAgent

agent = DQAgent(
    env,
    epsilon_max=max_epsilon,
    epsilon_min=min_epsilon,
    epsilon_decay=epsilon_decay,
    gamma=gamma,
    replay_memory_max_size=replay_memory_max_size,
    replay_memory_init_size=replay_memory_init_size,
    batch_size=batch_size,
    step_per_update=step_per_update,
    step_per_update_target_model=step_per_update_target_model,
    moving_avg_window_size=moving_avg_window_size,
    moving_avg_stop_thr=moving_avg_stop_thr,
    episode_max_steps=episode_max_steps,
)

In [8]:
import time

from training.dqlearning import DQLearning

train_start_time = time.time()

trainer = DQLearning(
    env,
    agent,
    episode_count=episode_count,
    episode_max_steps=episode_max_steps,
    dqn_action_model=action_net,
    dqn_target_model=target_net,
)
train_rewards = trainer.simple_dqn_training()

train_finish_time = time.time()
train_elapsed_time = train_finish_time - train_start_time
train_avg_episode_time = train_elapsed_time / episode_count

print(
    f"Train time: {train_elapsed_time / 60.0:.1f}m [{train_avg_episode_time:.1f}s]"
)

Training DQN:   0%|          | 0/500 [00:00<?, ?ep/s]

2025-10-20 16:32:28,874 — INFO — Episode: 0 | Steps: 18[18] | Epsilon: 1.000 | Time: 1.12s | Reward: 18.0 | MovingAvg: 18.0


Training DQN:   0%|          | 1/500 [00:01<09:21,  1.13s/ep]

2025-10-20 16:32:29,342 — INFO — Episode: 1 | Steps: 16[34] | Epsilon: 0.996 | Time: 0.47s | Reward: 16.0 | MovingAvg: 16.0


Training DQN:   0%|          | 2/500 [00:01<06:07,  1.35ep/s]

...

2025-10-20 16:40:43,201 — INFO — Episode: 187 | Steps: 141[6760] | Epsilon: 0.010 | Time: 13.83s | Reward: 141.0 | MovingAvg: 99.3


Training DQN:  38%|███▊      | 188/500 [08:15<56:12, 10.81s/ep]

2025-10-20 16:40:49,982 — INFO — Episode: 188 | Steps: 63[6823] | Epsilon: 0.010 | Time: 6.78s | Reward: 63.0 | MovingAvg: 101.8


Training DQN:  38%|███▊      | 188/500 [08:22<13:53,  2.67s/ep]

Train time: 8.4m [1.0s]


In [9]:
trainer.play_with_pygame(episodes=5, fps=30)

2025-10-20 16:42:55,614 — INFO — Episode 1/5 - Reward: 52.0
2025-10-20 16:43:02,864 — INFO — Episode 2/5 - Reward: 116.0
2025-10-20 16:43:06,640 — INFO — Episode 3/5 - Reward: 61.0
2025-10-20 16:43:10,553 — INFO — Episode 4/5 - Reward: 62.0
2025-10-20 16:43:14,240 — INFO — Episode 5/5 - Reward: 60.0
